# Lichess Atomic Analysis (Version 1.0)

**Ying Zhou**

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# 1. Obtain the list of files

In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [4]:
vari = 'atomic'

In [44]:
def obtain_slot_list (variant):
    r = requests.get('https://database.lichess.org')
    if r.status_code != 200:
        print(f"Error! The status code is {r.status_code}")
        return None
    soup = BeautifulSoup(r.content, 'html.parser')
    href_list = soup.find_all('a', href = re.compile(variant + '/lichess'))
    slot_list = ['https://database.lichess.org/' + href['href'] for href in href_list]
    return slot_list
    

In [45]:
slot_list = obtain_slot_list(vari)

In [46]:
slot_list

['https://database.lichess.org/atomic/lichess_db_atomic_rated_2019-02.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2019-01.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-12.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-11.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-10.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-09.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-08.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-07.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-06.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-05.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-04.pgn.bz2',
 'https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-03.pgn.bz2',
 'https://database.lichess.org/atomic/li

# 2. Obtain data

In [15]:
import bz2
import urllib.request
import os

Now we will write an extraction method that can extract stuff from files.

In [38]:
def obtain_pgn(variant, slot):
    pgn_file = variant + '_' + slot[-15:-8] + '.pgn'
    bz2_file = pgn_file + '.bz2'
    urllib.request.urlretrieve(slot, bz2_file)
    pgn_content = bz2.BZ2File(bz2_file).read().decode("utf-8")
    with open(pgn_file, 'w') as f:
        f.write(pgn_content)
    os.remove(bz2_file)
    return pgn_file

In [39]:
pgn_file = obtain_pgn(vari, slot_list[0])

In [23]:
import sys
!{sys.executable} -m pip install python-chess

  Using cached https://files.pythonhosted.org/packages/3e/f5/a00ac159d8970f6727861f58f8109a4783e32b63e65375c816de6a56ecca/python_chess-0.27.2-py3-none-any.whl


In [25]:
import chess
import chess.pgn

In [43]:
with open(pgn_file, 'r') as f:
    game = chess.pgn.read_game(f)
    print(game.headers["WhiteElo"])

1389


In [56]:
def extract_data(variant):
    slot_list = obtain_slot_list(variant)
    file_name = variant + '_' + slot_list[-1][-15:-8] + '_' + slot_list[0][-15:-8] + '.csv'
    with open(file_name, 'w') as g:
        g.write('wrat,brat,res,tc,tca\n')
        for slot in slot_list:
            print(f"Now processing {slot}\n")
            pgn_file = obtain_pgn(variant, slot)
            with open(pgn_file, 'r') as f:
                pgn = f.read()
                print(pgn[-3000:])
            with open(pgn_file, 'r') as f:
                while True:
                    game = chess.pgn.read_game(f)
                    if not game:
                        break
                    wrat = game.headers["WhiteElo"]
                    brat = game.headers["BlackElo"]
                    res = game.headers["Result"]
                    if res == '1-0':
                        nres = 1
                    elif res == '0-1':
                        nres = -1
                    elif res == '1/2-1/2':
                        nres = 0
                    else:
                        print(f"Result {res} is nonstandard!\n")
                        continue
                    tc = game.headers["TimeControl"]
                    tc_list = tc.split('+')
                    if len(tc_list) != 2:
                        print(f"TimeControl {tc} is nonstandard!\n")
                        continue
                    g.write(f"{wrat},{brat},{nres},{tc_list[0]},{tc_list[1]}\n")
            os.remove(pgn_file)
                
                    

In [55]:
extract_data('atomic')

Now processing https://database.lichess.org/atomic/lichess_db_atomic_rated_2019-02.pgn.bz2

"]
[Round "-"]
[White "SNovik1995"]
[Black "chycha"]
[Result "0-1"]
[UTCDate "2019.02.28"]
[UTCTime "22:59:47"]
[WhiteElo "1252"]
[BlackElo "1736"]
[WhiteRatingDiff "-10"]
[BlackRatingDiff "+2"]
[TimeControl "300+0"]
[Termination "Normal"]
[Variant "Atomic"]

1. e4 { [%clk 0:05:00] } Nf6 { [%clk 0:05:00] } 2. f3 { [%clk 0:04:54] } d5 { [%clk 0:04:58] } 3. Bb5+ { [%clk 0:04:47] } c6 { [%clk 0:04:56] } 4. exd5 { [%clk 0:04:46] } Qxd2# { [%clk 0:04:54] } 0-1

[Event "Rated Atomic game"]
[Site "https://lichess.org/UubwwCBe"]
[Date "2019.02.28"]
[Round "-"]
[White "shadowschuh6"]
[Black "AtomicRhino"]
[Result "0-1"]
[UTCDate "2019.02.28"]
[UTCTime "22:59:52"]
[WhiteElo "1843"]
[BlackElo "2164"]
[WhiteRatingDiff "-3"]
[BlackRatingDiff "+4"]
[TimeControl "0+1"]
[Termination "Time forfeit"]
[Variant "Atomic"]

1. h4? { [%eval -1.79] [%clk 0:00:03] } 1... e6? { [%eval 0.25] [%clk 0:00:03] } 2. e3 { [%eva

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!



error during pgn parsing
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/chess/__init__.py", line 2696, in parse_san
    return next(move for move in self.generate_castling_moves() if self.is_queenside_castling(move))
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/chess/pgn.py", line 1249, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "/anaconda3/lib/python3.7/site-packages/chess/pgn.py", line 693, in parse_san
    return board.parse_san(san)
  File "/anaconda3/lib/python3.7/site-packages/chess/__init__.py", line 2698, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'O-O-O' in r7/8/2p5/p2p4/3P3q/2P5/4kR2/R3K2q w Q - 0 41


Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Result * is nonstandard!

Now processing https://database.lichess.org/atomic/lichess_db_atomic_rated_2018-11.pgn.bz2

1661"]
[WhiteRatingDiff "+9"]
[BlackRatingDiff "-13"]
[TimeControl "180+0"]
[Termination "Normal"]
[Variant "Atomic"]

1. Nf3 { [%clk 0:03:00] } e5 { [%clk 0:03:00] } 2. Ng5 { [%clk 0:02:56] } f5 { [%clk 0:02:59] } 3. d4 { [%clk 0:02:40] } e4 { [%clk 0:02:49] } 4. e3 { [%clk 0:02:19] } Nh6 { [%clk 0:02:42] } 5. Nf7 { [%clk 0:01:55] } Qh4 { [%clk 0:02:36] } 6. g3 { [%clk 0:01:53] } Qf6 { [%clk 0:02:21] } 7. Ne5 { [%clk 0:01:44] } d5 { [%clk 0:02:15] } 8. Nd7 { [%clk 0:01:33] } Bb4+ { [%clk 0:02:08] } 9. c3 { [%clk 0:01:30] } Ng4 { [%clk 0:02:03] } 10. f4 { [%clk 0:01:13] } Nf2 { [%clk 0:01:59] } 11. Qh5+ { [%clk 0:01:07] } Qg6 { [%clk 0:01:50] } 12. Nxb8 { [%clk 0:00:52] } Nd1 { [%clk 0:01:44] } 13. Qxh7 { [%clk 0:00:42] } Nxb2 { [%clk 0:01:43

error during pgn parsing
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/chess/__init__.py", line 2694, in parse_san
    return next(move for move in self.generate_castling_moves() if self.is_kingside_castling(move))
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/chess/pgn.py", line 1249, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "/anaconda3/lib/python3.7/site-packages/chess/pgn.py", line 693, in parse_san
    return board.parse_san(san)
  File "/anaconda3/lib/python3.7/site-packages/chess/__init__.py", line 2698, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'O-O' in 3Qk2r/p2K1ppp/2n1rn2/1p6/8/5N2/PP4PP/R1B4R b k - 3 20


...

error during pgn parsing
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/chess/pgn.py", line 1249, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "/anaconda3/lib/python3.7/site-packages/chess/pgn.py", line 693, in parse_san
    return board.parse_san(san)
  File "/anaconda3/lib/python3.7/site-packages/chess/__init__.py", line 2744, in parse_san
    raise ValueError("illegal san: {!r} in {}".format(san, self.fen()))
ValueError: illegal san: 'dxe5' in r2k1b1r/ppNBq1pp/5p1n/2p1p3/3P4/8/PPP3PP/RNBQK2R w KQ - 4 12
error during pgn parsing
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/chess/pgn.py", line 1249, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "/anaconda3/lib/python3.7/site-packages/chess/pgn.py", line 693, in parse_san
    return board.parse_san(san)
  File "/anaconda3/lib/python3.7/site-packages/chess/__init__.py", line 2744, in parse_san
    raise ValueErr

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeCont

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeCont

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeCont


TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeControl - is nonstandard!

TimeCon

# 3. Preliminary Analysis

In [57]:
df = pd.read_csv('atomic_2015-01_2019-02.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
df.tail()

,wrat,brat,res,tc,tca
3570526,1500,1566,-1,120,0
3570527,1773,1631,1,900,9
3570528,1903,1883,-1,180,0
3570529,1870,1906,1,180,0
3570530,1500,1595,-1,120,0


In [74]:
wrat_list = df['wrat'].unique().tolist()

In [77]:
new_wrat_list = list(map(float, wrat_list))

ValueError: could not convert string to float: '?'

In [72]:
wrat_list.sort()

In [75]:
wrat_lis

[1389,
 2406,
 1364,
 2067,
 1681,
 2384,
 2059,
 2392,
 1344,
 1338,
 2146,
 2053,
 2400,
 1330,
 1922,
 1396,
 2046,
 1454,
 2147,
 1483,
 1808,
 1599,
 2407,
 1359,
 1351,
 1061,
 1941,
 1927,
 1579,
 2040,
 1538,
 1411,
 1817,
 2414,
 1490,
 1943,
 1584,
 1328,
 1498,
 1729,
 1108,
 1415,
 1911,
 1477,
 1913,
 1841,
 1587,
 1343,
 1412,
 1349,
 1442,
 1300,
 1988,
 1842,
 1562,
 1347,
 1537,
 1478,
 1192,
 1945,
 1336,
 1489,
 1474,
 1575,
 1323,
 1362,
 1972,
 1869,
 1661,
 1682,
 1982,
 1507,
 1331,
 1802,
 1589,
 1850,
 1632,
 1804,
 1235,
 1406,
 1871,
 1807,
 1849,
 2109,
 1824,
 2118,
 1524,
 2621,
 1816,
 1830,
 2354,
 2026,
 1868,
 2012,
 1633,
 1429,
 2073,
 2000,
 2635,
 1650,
 1955,
 1838,
 2120,
 2365,
 1500,
 1668,
 2007,
 1526,
 1425,
 1861,
 2088,
 1997,
 1619,
 1417,
 1846,
 2611,
 1558,
 1863,
 2358,
 2121,
 1878,
 1735,
 2131,
 2044,
 1434,
 1969,
 1667,
 2141,
 2608,
 1987,
 1511,
 2096,
 2010,
 2360,
 1746,
 1519,
 1437,
 1237,
 1710,
 1833,
 1528,
 1287,
 2014,

In [61]:
df_high = df[(df['wrat'] >= 2400) & (df['brat'] >= 2400)]

TypeError: '>=' not supported between instances of 'str' and 'int'

It seems that we have NaN here.

In [78]:
df_nan = df.applymap(np.isreal)

In [85]:
df_nan.tca.value_counts()

True    3570531
Name: tca, dtype: int64

In [90]:
df.wrat.value_counts()['?']

1

In [91]:
df_num_temp = df[df.wrat.apply(lambda x: x.isnumeric())]
df_num = df_num_temp[df_num_temp.brat.apply(lambda x: x.isnumeric())]

AttributeError: 'int' object has no attribute 'isnumeric'

In [95]:
df.wrat.sort_values()

TypeError: '<' not supported between instances of 'int' and 'str'

In [101]:
wrats = df['wrat'].apply(str).sort_values().unique().tolist()
brats = df['brat'].apply(str).sort_values().unique().tolist()

In [102]:
for r in wrats:
    print(f"{r}")

1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199


In [103]:
for r in brats:
    print(f"{r}")

1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199


In [104]:
def convert(rat):
    try:
        a = int(rat)
        return a
    except ValueError:
        return np.nan

In [108]:
convert('strg')

nan

In [111]:
df['wrat'] = df.wrat.apply(convert)
df['brat'] = df.brat.apply(convert)

In [112]:
df.head()

,wrat,brat,res,tc,tca
0,1389.0,1350.0,1,180,2
1,2406.0,2045.0,-1,30,0
2,1364.0,1447.0,1,1200,0
3,2067.0,2380.0,-1,30,0
4,1681.0,1400.0,1,180,2


In [113]:
df_num = df.dropna()

In [114]:
df_num.shape

(3570529, 5)

In [115]:
df.shape

(3570531, 5)

In [116]:
df_num.dtypes

wrat    float64
brat    float64
res       int64
tc        int64
tca       int64
dtype: object

In [118]:
df_high = df_num[(df_num['wrat'] > 2399.5) & (df_num['brat'] > 2399.5)]

In [119]:
df_high.shape

(14877, 5)

In [120]:
df_long = df_high[df_high['tc'] >= 180]

In [121]:
df_long.shape

(5002, 5)

In [124]:
s_v = df_long.res.value_counts()

In [126]:
s_v = s_v / df_long.shape[0]

In [127]:
s_v

 1    0.558976
-1    0.338465
 0    0.102559
Name: res, dtype: float64

In [128]:
df_high.res.value_counts()/df_high.shape[0]

 1    0.550044
-1    0.383478
 0    0.066478
Name: res, dtype: float64

In [129]:
df_rlong = df_high[df_high['tc'] >= 300]

In [130]:
df_rlong.shape

(563, 5)

In [131]:
df_rlong.res.value_counts()/df_rlong.shape[0]

 1    0.509769
-1    0.383659
 0    0.106572
Name: res, dtype: float64

In [132]:
df.res.value_counts()/df.shape[0]

 1    0.540386
-1    0.440616
 0    0.018998
Name: res, dtype: float64

In [133]:
df_moderate = df_num[(df_num['wrat'] > 1999.5) & (df_num['brat'] > 1999.5)]

In [134]:
df_moderate.res.value_counts()/df_moderate.shape[0]

 1    0.554146
-1    0.407083
 0    0.038771
Name: res, dtype: float64

In [135]:
df_mlong = df_moderate[df_moderate['tc'] >= 180]

In [136]:
df_moderate.shape

(471206, 5)

In [137]:
df_mlong.shape

(115690, 5)

In [138]:
df_mlong.res.value_counts()/df_mlong.shape[0]

 1    0.556842
-1    0.376031
 0    0.067128
Name: res, dtype: float64